In [1]:
#Import Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import preprocessing


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, explained_variance_score,mean_absolute_error,mean_squared_error
from math import sqrt
from sklearn.model_selection import RandomizedSearchCV,StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score,explained_variance_score,confusion_matrix,accuracy_score,classification_report,log_loss

#To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

#Increase the size of sns plots
sns.set(rc={'figure.figsize':(5,5)})

#View all the dataframe
pd.set_option('display.max_columns',None)


In [2]:
heart_df = pd.read_csv("~/Downloads/heart.csv")

heart_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


# EDA & PROCESSING

In [4]:
#examine information in dataframe
heart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [5]:
#Check datatypes
heart_df.dtypes

Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object

In [6]:
#examine total Nan values for each column
heart_df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [7]:
#Determine number of unique values for each cloumn
heart_df.nunique()

Age                50
Sex                 2
ChestPainType       4
RestingBP          67
Cholesterol       222
FastingBS           2
RestingECG          3
MaxHR             119
ExerciseAngina      2
Oldpeak            53
ST_Slope            3
HeartDisease        2
dtype: int64

In [8]:
#Normalize data

new_raw_data = pd.get_dummies(heart_df,columns =['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope'])
new_raw_data.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [9]:
#Change column names
new_raw_data.rename(columns={'Sex_0': 'Male', 'Sex_1': 'Female','ChestPainType_0':'ChestPainType_ASY','ChestPainType_1':'ChestPainType_ATA','ChestPainType_2':'ChestPainType_NAP','ChestPainType_3':'ChestPainType_TA','RestingECG_0':'RestingECG_LVH','RestingECG_1':'RestingECG_Normal','RestingECG_2':'RestingECG_ST','ExerciseAngina_0':'ExerciseAngina_N','ExerciseAngina_1':'ExerciseAngina_Y','ST_Slope_0':'ST_Slope_Down','ST_Slope_1':'ST_Slope_Flat','ST_Slope_2':'ST_Slope_Up'}, inplace=True)
new_raw_data.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [27]:
#Look at Heart disease outcome value count
heart_disease_count = new_raw_data['HeartDisease'].value_counts()
heart_disease_count

1    508
0    410
Name: HeartDisease, dtype: int64

In [28]:
#Split data into labels and features
y=new_raw_data['HeartDisease']

X=new_raw_data.drop(columns= ['HeartDisease'])

In [30]:
#Review the y vaviable series
y.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [31]:
#Review the X variable DataFrame
X.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [32]:
#Check balance of target heartdisease value
y.value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [35]:
 #Split processed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)


In [36]:
#Create standard sclaer instance
scaler= StandardScaler()

In [38]:
#Fit the standardScaler with training data
X_scaler = scaler.fit(X_train)

In [39]:
#Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# TRAIN AND EVALUATE

In [21]:
#Create Randomforestclassifier model

In [77]:
import warnings 
warnings.filterwarnings("ignore")

In [78]:
# Import Dependencies
from imblearn.ensemble import BalancedRandomForestClassifier


# Instantiate the randomforestclassifier model
# Assign a random_state parameter of 1 to the model
classifier = BalancedRandomForestClassifier (n_estimators=100,random_state=1)

# Fit the model using training data
classifier.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=1)

In [50]:
# Make a prediction using the testing data
predictions= classifier.predict(X_test)

In [69]:
#Display confused matix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[ 65,   9],
       [ 10, 100]])

In [70]:
# Evaluate the model with Confusion Matrix and accuracy score
c_matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    c_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

accu_score = accuracy_score(y_test, predictions)


In [71]:
#Generate matrix model
display(cm_df)

,Predicted 0,Predicted 1
Actual 0,65,9
Actual 1,10,100


In [72]:
#calculate balanced score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test,predictions)

0.8937346437346437

In [67]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))


                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.88      0.91      0.87      0.89      0.80        74
          1       0.92      0.91      0.88      0.91      0.89      0.80       110

avg / total       0.90      0.90      0.89      0.90      0.89      0.80       184



In [68]:
print ("Classification Report")
print(classification_report(y_test,predictions))

Classification Report
              precision    recall  f1-score   support

           0       0.87      0.88      0.87        74
           1       0.92      0.91      0.91       110

    accuracy                           0.90       184
   macro avg       0.89      0.89      0.89       184
weighted avg       0.90      0.90      0.90       184

